In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone


In [ ]:
TABLE_PATH = src.config.BRONZE / "wh_acctcommon"

In [ ]:
wh_acctcommon = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
wh_acctcommon = wh_acctcommon[wh_acctcommon['mjaccttypcd'].isin(['CK','SAV','TD'])].copy()

In [ ]:
wh_acctcommon['curracctstatcd'].unique()

In [ ]:
wh_acctcommon

In [ ]:
TABLE_PATH = src.config.SILVER / "account"

In [ ]:
history = DeltaTable(TABLE_PATH).history()
history_df = pd.DataFrame(history)


In [ ]:
def convert_ts(ts):
    if len(str(ts)) == 13:
        return pd.to_datetime(ts, unit='ms', utc=True)
    else:
        return pd.to_datetime(ts, unit='s', utc=True)

history_df['readable_time'] = history_df['timestamp'].apply(convert_ts)

In [ ]:
history_df['eastern_time'] = history_df['readable_time'].dt.tz_convert('America/New_York')

In [ ]:
history_df

In [ ]:
df = DeltaTable(TABLE_PATH, version=8).to_pandas()

In [ ]:
df['eastern_time'] = df['load_timestamp_utc'].dt.tz_convert('America/New_York')

In [ ]:
df

In [ ]:
deposits = df[df['mjaccttypcd'].isin(['SAV','CK','TD'])].copy()

In [ ]:
dep_total = deposits['notebal'].sum()
print(f"Deposit Total: ${dep_total:,.2f}")

In [ ]:
deposits

In [ ]:
muni_slice_criteria = (deposits['product'].str.contains('muni',case=False,na=False)) & (~deposits['product'].str.contains('Community',case=False,na=False))
muni_slice = deposits[muni_slice_criteria].copy()

Product:
- contains muni
- does not contain 'Community'

In [ ]:
muni_total = muni_slice['notebal'].sum()
print(f"Muni Total: ${muni_total:,.2f}")

In [ ]:
# muni_slice['product'].unique()

In [ ]:
deposits_other = deposits[~muni_slice_criteria].copy()

In [ ]:
deposits_other['notebal'].sum()

In [ ]:
deposits_other['product'].unique()

In [ ]:
business_criteria = (deposits_other['product'].str.contains('Business',case=False, na=False)) | (deposits_other['product'].str.contains('IOLTA',case=False,na=False)) | (deposits_other['product'].str.contains('Community',case=False,na=False)) | (deposits_other['product'].str.contains('Commercial',case=False,na=False)) | (deposits_other['product'].str.contains('BCSB Internal Holdback Savings',case=False,na=False))

Business:
- name contains 'business'
- name contains 'iolta'
- name contains 'community'
- name contains 'commercial'
- name contains 'BCSB Internal Holdback Savings'

In [ ]:
business_slice = deposits_other[business_criteria].copy()

In [ ]:
business_slice['notebal'].sum()

In [ ]:
business_slice

In [ ]:
region_mapping = {
    ''
}
0	BCSB - ATTLEBORO BRANCH
1	BCSB - ATTLEBORO HIGH SCHOOL
2	BCSB - CANDLEWORKS BRANCH
3	BCSB - CONTACT CENTER
4	BCSB - COUNTY STREET BRANCH
5	BCSB - CUMBERLAND
6	BCSB - DARTMOUTH BRANCH
7	BCSB - DEPOSIT OPERATIONS
8	BCSB - EAST FREETOWN BRANCH
9	BCSB - FALL RIVER BRANCH
10	BCSB - FRANKLIN BRANCH
11	BCSB - GREENVILLE
12	BCSB - MAIN OFFICE
13	BCSB - NB ASHLEY BLVD BRANCH
14	BCSB - NO ATTLEBORO BRANCH
15	BCSB - NORTH RAYNHAM BRANCH
16	BCSB - PAWTUCKET BRANCH
17	BCSB - RAYNHAM CENTER BRANCH
18	BCSB - REHOBOTH BRANCH
19	BCSB - TAUNTON HIGH SCHOOL

urrent Structure on Markets:

South Coast Region:
                                                              i.      New Bedford – Downtown

                                                             ii.      New Bedford – Ashley Blvd

                                                           iii.      Dartmouth

                                                           iv.      E. Freetown

                                                             v.      Fall River

Greater Attleboro Region:
                                                           vi.      Attleboro

                                                         vii.      N. Attleboro

                                                        viii.      Rehoboth

                                                           ix.      Franklin

Taunton Region:
                                                             x.      Taunton (main office)

                                                           xi.      Raynham Center

                                                          xii.      N. Raynham

                                                        xiii.      County Street

 

RI Region:
                                                        xiv.      Pawtucket

                                                          xv.      Greenville

                                                        xvi.      Cumberland

                                                      xvii.      Providence (potential 2025) (SBRM’s working market)


In [ ]:
region_mapping = {
    'BCSB - ATTLEBORO BRANCH': 'Greater Attleboro',
    'BCSB - ATTLEBORO HIGH SCHOOL': 'Greater Attleboro',  # High school in Attleboro
    'BCSB - CANDLEWORKS BRANCH': 'Unknown',  # Not listed in structure; assume or research to assign
    'BCSB - CONTACT CENTER': 'Taunton',  # Central to Taunton Region per Main Office
    'BCSB - COUNTY STREET BRANCH': 'Taunton',
    'BCSB - CUMBERLAND': 'RI',
    'BCSB - DARTMOUTH BRANCH': 'South Coast',
    'BCSB - DEPOSIT OPERATIONS': 'Taunton',  # Central to Taunton Region per Main Office
    'BCSB - EAST FREETOWN BRANCH': 'South Coast',
    'BCSB - FALL RIVER BRANCH': 'South Coast',
    'BCSB - FRANKLIN BRANCH': 'Greater Attleboro',
    'BCSB - GREENVILLE': 'RI',
    'BCSB - MAIN OFFICE': 'Taunton',
    'BCSB - NB ASHLEY BLVD BRANCH': 'South Coast',
    'BCSB - NO ATTLEBORO BRANCH': 'Greater Attleboro',
    'BCSB - NORTH RAYNHAM BRANCH': 'Taunton',
    'BCSB - PAWTUCKET BRANCH': 'RI',
    'BCSB - RAYNHAM CENTER BRANCH': 'Taunton',
    'BCSB - REHOBOTH BRANCH': 'Greater Attleboro'
}

In [ ]:
not_business = deposits_other[~business_criteria].copy()

In [ ]:
not_business['product'].unique()

In [ ]:
not_business['notebal'].sum()

In [ ]:
not_business

In [ ]:
muni

In [ ]:
muni['PRODUCT'].unique()

In [ ]:
muni['NOTEBAL'].sum()